In [223]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [594]:
df = pd.read_csv('clean_datafile.csv')
df.head(3)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,plot_keywords,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,avatar|future|marine|native|paraplegic,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,goddess|marriage ceremony|marriage proposal|pi...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,bomb|espionage|sequel|spy|terrorist,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,85000


In [595]:
df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'plot_keywords', 'num_user_for_reviews', 'language',
       'country', 'content_rating', 'budget', 'title_year',
       'actor_2_facebook_likes', 'imdb_score', 'movie_facebook_likes'],
      dtype='object')

### Lets choose the most relevent columns

In [596]:
df=df[['movie_title',
       'genres', 
       'director_name', 
       'actor_1_name', 
       'actor_2_name', 
       'actor_3_name', 
       'plot_keywords',
       'language',
       'country','title_year', 
       'imdb_score',
       'color',
       'content_rating']]
df.head(5)

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,plot_keywords,language,country,title_year,imdb_score,color,content_rating
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar|future|marine|native|paraplegic,English,USA,2009.0,7.9,Color,PG-13
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,English,USA,2007.0,7.1,Color,PG-13
2,Spectre,Action|Adventure|Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,English,UK,2015.0,6.8,Color,PG-13
3,The Dark Knight Rises,Action|Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,English,USA,2012.0,8.5,Color,PG-13
4,John Carter,Action|Adventure|Sci-Fi,Andrew Stanton,Daryl Sabara,Samantha Morton,Polly Walker,alien|american civil war|male nipple|mars|prin...,English,USA,2012.0,6.6,Color,PG-13


In [597]:
df=df.astype(str)

In [598]:
x=['genres','plot_keywords']
for column in x:
    df[column] = df[column].map(lambda x: x.lower().split('|'))  #Action|Adventure|Fantasy|Sci-Fi ---->[action, adventure, fantasy, sci-fi]
    df[column]=df[column].str.join(' ')                       #[action, adventure, fantasy, sci-fi]---->action adventure fantasy sci-fi

In [599]:
df.head(3)

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,plot_keywords,language,country,title_year,imdb_score,color,content_rating
0,Avatar,action adventure fantasy sci-fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar future marine native paraplegic,English,USA,2009.0,7.9,Color,PG-13
1,Pirates of the Caribbean: At World's End,action adventure fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,goddess marriage ceremony marriage proposal pi...,English,USA,2007.0,7.1,Color,PG-13
2,Spectre,action adventure thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,bomb espionage sequel spy terrorist,English,UK,2015.0,6.8,Color,PG-13


In [600]:
y=['actor_1_name', 'actor_2_name', 'actor_3_name','director_name']
for col in y:
    df[col] = df[col].map(lambda x: x.split(' ')) # James Cameron       ------->  ['James','Cameron']
    df[col] = df[col].str.join('')                # ['James','Cameron'] ------->  JamesCameron

In [601]:
df.head(3)

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,plot_keywords,language,country,title_year,imdb_score,color,content_rating
0,Avatar,action adventure fantasy sci-fi,JamesCameron,CCHPounder,JoelDavidMoore,WesStudi,avatar future marine native paraplegic,English,USA,2009.0,7.9,Color,PG-13
1,Pirates of the Caribbean: At World's End,action adventure fantasy,GoreVerbinski,JohnnyDepp,OrlandoBloom,JackDavenport,goddess marriage ceremony marriage proposal pi...,English,USA,2007.0,7.1,Color,PG-13
2,Spectre,action adventure thriller,SamMendes,ChristophWaltz,RoryKinnear,StephanieSigman,bomb espionage sequel spy terrorist,English,UK,2015.0,6.8,Color,PG-13


In [602]:
df['bag_of_words']=df['director_name'].astype(str)+" "+ df['actor_2_name'].astype(str)+" "+df['genres'].astype(str)+" "+df['actor_1_name'].astype(str)+" "+df['actor_3_name'].astype(str)+" "+df['plot_keywords'].astype(str)+" "+df['language'].astype(str)+" "+df['country'].astype(str)+" "+df['content_rating'].astype(str)

In [603]:
df.drop(columns = ['genres', 'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'plot_keywords', 'language','country','title_year', 'imdb_score','color','content_rating'], inplace = True)

In [604]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])


In [607]:
df.head(3)

,movie_title,bag_of_words
0,Avatar,JamesCameron JoelDavidMoore action adventure f...
1,Pirates of the Caribbean: At World's End,GoreVerbinski OrlandoBloom action adventure fa...
2,Spectre,SamMendes RoryKinnear action adventure thrille...


In [608]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.36893239, 0.29462783, ..., 0.14213381, 0.27216553,
        0.14744196],
       [0.36893239, 1.        , 0.2795085 , ..., 0.13483997, 0.25819889,
        0.13987572],
       [0.29462783, 0.2795085 , 1.        , ..., 0.15075567, 0.21650635,
        0.10425721],
       ...,
       [0.14213381, 0.13483997, 0.15075567, ..., 1.        , 0.34815531,
        0.12573892],
       [0.27216553, 0.25819889, 0.21650635, ..., 0.34815531, 1.        ,
        0.18057878],
       [0.14744196, 0.13987572, 0.10425721, ..., 0.12573892, 0.18057878,
        1.        ]])

In [613]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = df[df['movie_title'].str.match(title)].index[0]
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 13 most similar movies
    top_10_indexes = list(score_series.iloc[1:13].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.movie_title)[i] )
        
    return recommended_movies

In [614]:
recommendations('Looper')

['The Butterfly Effect\xa0',
 'Source Code\xa0',
 'The Time Machine\xa0',
 'Timecop\xa0',
 'Clockstoppers\xa0',
 'X-Men: Days of Future Past\xa0',
 'Donnie Darko\xa0',
 'A Sound of Thunder\xa0',
 'Terminator 2: Judgment Day\xa0',
 'Back to the Future\xa0',
 'Chappie\xa0',
 'In Time\xa0']

###  Looper
Looper is basically time travel movie ,so is recommended movies.